In [ ]:
import numpy as np
import os
from sympy.solvers import solve
from sympy import Symbol
from sklearn.metrics import accuracy_score

In [ ]:
model_1_path = os.path.join(os.getcwd(), "Confidence Score", "MHEALTH", "MHEALTH_incep.npy")
model_2_path = os.path.join(os.getcwd(), "Confidence Score", "MHEALTH", "MHEALTH_resnet.npy")
model_3_path = os.path.join(os.getcwd(), "Confidence Score", "MHEALTH", "MHEALTH_cus.npy")
model_4_path = os.path.join(os.getcwd(), "Confidence Score", "MHEALTH", "MHEALTH_den.npy")
model_5_path = os.path.join(os.getcwd(), "Confidence Score", "MHEALTH", "MHEALTH_xcep.npy")

test_path = os.path.join(os.getcwd(), "Confidence Score", "MHEALTH", "MHEALTH_test.npz")

In [ ]:
model_1_score=np.load(model_1_path)
model_2_score=np.load(model_2_path)
model_3_score=np.load(model_3_path)
model_4_score=np.load(model_4_path)
model_5_score=np.load(model_5_path)

y_test=np.load(test_path)
y_test=y_test['y_test']

In [ ]:
def Compute_A_3(f,l):
    n = len(f)
    temp_1 = f[0]+f[1]+f[2]
    temp_2 = l * ((f[0] * f[1]) + (f[0] * f[2]) + (f[1] * f[2]))
    temp_3 = (f[0] * f[1] * f[2]) * (l ** (n-1))
    
    return (temp_1 + temp_2 + temp_3)

In [ ]:
def Compute_A_4(f,l):
    n = len(f)
    temp_1 = f[0]+f[1]+f[2] + f[3]
    temp_2 = l * ((f[0] * f[1]) + (f[0] * f[2]) + (f[0] * f[3]) + (f[1] * f[2]) + (f[1] * f[3]) + (f[2] * f[3]))
    temp_3 = (f[0] * f[1] * f[2] * f[3]) * (l ** (n-1))
    
    return (temp_1 + temp_2 + temp_3)

In [ ]:
fuzzy_measure=np.array([0.23463945081459225, 0.13328061612163145, 0.03364719889913448, 0.2610887108305966, 0.03801994640207629])

In [ ]:
l=Symbol('l',real=True)
lam=solve( ( (1+l*fuzzy_measure[0])*(1+l*fuzzy_measure[1])*(1+l*fuzzy_measure[2])*(1+l*fuzzy_measure[3])*(1+l*fuzzy_measure[4]) )-(1+l) , l )
lam=float(lam[2])
lam

In [ ]:
fuzzy_score=np.zeros(shape=model_1_score.shape, dtype=float)
fuzzy_score.shape

In [ ]:
for i in range(fuzzy_score.shape[0]):
    for j in range(6):
        
        f_score=np.asarray([ model_1_score[i][j], model_2_score[i][j], model_3_score[i][j], model_4_score[i][j], model_5_score[i][j] ])
        
        permute_index=np.argsort(f_score)
        f_score_permuted=f_score[permute_index]
        fuzzy_measure_permuted=fuzzy_measure[permute_index]
        
        A_0=1.0
        A_1 =Compute_A_4(fuzzy_measure_permuted[1:], lam)
        A_2 = Compute_A_3(fuzzy_measure_permuted[2:], lam)
        A_3=( fuzzy_measure_permuted[3] + fuzzy_measure_permuted[4] + lam*fuzzy_measure_permuted[3]*fuzzy_measure_permuted[4] )
        A_4=fuzzy_measure_permuted[4]
        #print("The A_i values are {} , {}, {}".format(A_0,A_1,A_2))
        
        sugeno_score=max([ min(f_score_permuted[0],A_0), min(f_score_permuted[1],A_1), min(f_score_permuted[2],A_2), min(f_score_permuted[3],A_3), min(f_score_permuted[4],A_4) ])
        
        fuzzy_score[i][j]=sugeno_score

In [ ]:
fuzzy_y_pred=np.argmax(fuzzy_score,axis=1)